In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
base_url = "https://www.expat.com"
url = "https://www.expat.com/en/jobs/middle-east/oman/"

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0"
}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

In [5]:
jobs = []

for box in soup.find_all("div", class_="item-box"):
    job = {}
    
    title_tag = box.find("a", class_="item-box__title")
    job["Title"] = title_tag.text.strip() if title_tag else "N/A"
    
    company_div = box.find("div", class_="item-box__user")
    job["Company"] = company_div.text.strip() if company_div else "N/A"

    misc = box.find("div", class_="item-box__misc")
    if misc:
        # Extraction de tous les textes visibles, même s'ils ne sont pas dans des <div>
        misc_texts = [x.strip() for x in misc.stripped_strings]
        
        job["Location"] = misc_texts[0] if len(misc_texts) > 0 else "N/A"
        
        # Recherche plus sûre de la date
        job["Date Posted"] = "N/A"
        for text in misc_texts:
            if "Added on" in text:
                job["Date Posted"] = text.replace("Added on", "").strip()
                break
    else:
        job["Location"] = "N/A"
        job["Date Posted"] = "N/A"

    # Gestion du lien
    if title_tag and title_tag.has_attr("href"):
        relative_url = title_tag["href"]
        job["Link"] = base_url + relative_url if relative_url.startswith("/") else relative_url
    else:
        job["Link"] = "N/A"

    # Accès à la page détaillée
    if job["Link"] != "N/A":
        detail_resp = requests.get(job["Link"], headers=headers)
        detail_soup = BeautifulSoup(detail_resp.content, "html.parser")
        description_div = detail_soup.find("div", class_="block-description")
        job["Description"] = description_div.get_text(separator=" ").strip() if description_div else "N/A"
    else:
        job["Description"] = "N/A"

    jobs.append(job)

In [6]:
for job in jobs:
    print(job)

{'Title': 'House maid needed in muscat Oman', 'Company': 'Asad', 'Link': 'https://www.expat.com/en/jobs/middle-east/oman/muscat/88-housemaid/700250-house-maid-needed-in-muscat-oman.html', 'Location': 'Muscat', 'Date Posted': '20/05/2025', 'Description': "Description \n \n \n \n Hi We are looking for a maid for our family. We live in muscat, Oman. The maid should be Indonesian. Practicing Muslim and have experience in cooking, cleaning the who house duty's (driving a car would be preferred but not essential). Must be able to speak either English or Arabic. Also willing and eager to learn. We will offer accommodation, return flight tickets once a year and salary of 150 OMR per month. Please get in touch if interested. Thanks Asad"}
{'Title': 'House Maid for family - Must be Indonesian', 'Company': 'Asad', 'Link': 'https://www.expat.com/en/jobs/middle-east/oman/88-housemaid/698667-house-maid-for-family---must-be-indonesian.html', 'Location': 'Muscat', 'Date Posted': '15/03/2025', 'Descrip